# Kefir community simulation using SteadierCom

In this tutorial you will learn you use a community simulation tool to analyse
metabolic cross-feeding interactions in a microbial community.

Our case-study will be the kefir community from [Blasche et al, 2021](https://www.nature.com/articles/s41564-020-00816-5).

We will try to find potential cross-feeding interactions between the four most abundant species during milk fermentation:

- *Lactobacillus kefiranofaciens*
- *Leuconocost mesenteroides*
- *Lactococcus lactis*
- *Acetobacter fabarum*

![fig1bd](images/fig1bd.png)

We will use a new simulation tool called [SteadierCom](https://github.com/cdanielmachado/steadiercom) (still under development), that we discussed during the theoretical part of this course. 

## 1. Running SteadierCom

SteadierCom is a command-line tool. We can use command-line tools in Jupyter (as if we were typing directly on a terminal) by starting our code with `!`.

Let's start with testing the help menu:

In [ ]:
! steadiercom -h

**SteadierCom** will require several inputs:

- Genome-scale metabolic models for each species
- Relative abundances of species (optional)
- Medium composition
- Community growth rate (optional)
- "Unlimited" compounds (like water, minerals, trace metals) to exclude from cross-feeding analysis (optional)
- Sample size (for sampling the solution space)

All these data has been prepared in the expected format (you can inspect it in the *data* folder). The growth rate and relative abundances have been grossly estimated from the figure above (assuming 25-fold total biomass increase in 90 hours).

We are now ready to run (this can take a few minutes):

In [ ]:
! steadiercom models/*.xml -c data/abundance.tsv -m MILK --mediadb data/milk_composition.tsv --growth 0.05 --unlimited data/unlimited.txt --sample 100

(When the simulation is finished you should see a new file called `output.tsv`.)

## 2. Analysing results

The output of **SteadierCom** is a table in TSV format that you can inspect with a text editor or open in a spreadsheet software.

In this tutorial, we will use the powerful [**pandas**](https://pandas.pydata.org/) library. If you never used **pandas** before, please take a few minutes to get familiar with the [documentation](https://pandas.pydata.org/docs/user_guide/index.html).

Let's start by loading and inspecting the file:

In [ ]:
import pandas as pd

df = pd.read_csv('output.tsv', sep='\t')

df.head(20) # show the first 20 lines

We can use the `.query()` and `.sort()` methods to find and sort entries.

Let's check who is consuming compounds from the environment, and sort them by consumption rate.

In [ ]:
df.query('donor == "environment"').sort_values('mass_rate', ascending=False)

### 2.1 - Inspecting interactions between species 

Let's first check for the main interactions highlighted in Fig. 5e the paper. 

![fig5e1](images/fig5e1.png)

*L. kefiranofaciens* "gives" amino acids to *L. mesenteroides* (although this is due to extracellular proteolytic activity and not "direct" cross-feeding). 

Can you select all cross-feeding interactions from *L. kefiranofaciens* to *L. mesenteroides* with a frequency of at least 0.1 ? 

> This time you will try by yourself! 🙂


In [ ]:
# type your code here...

Or click below to reveal the solution:

In [ ]:

df.query('donor == "L_kefiranofaciens" and receiver == "L_mesenteroides" and frequency > 0.1')

Since these simulations are based on random sampling of the solution space, 
it is possible that your results are different from the teacher or from your colleagues.

But, most likely, your result was an empty table. 

Now let's inspect what is possibly donated from *L. mesenteroides* to *L. kefiranofaciens*:

In [ ]:
# type your code here...

Or click below to reveal the solution:

In [ ]:

df.query('receiver == "L_kefiranofaciens" and donor == "L_mesenteroides" and frequency > 0.1' )

L. mesenteroides is expected to donate lactate to L. kefiranofaciens. In the simulations we observe exchange of acetaldehyde instead (this is likely due to a common issue with the models generated with CarveMe, they often ferment acetaldehyde instead of acetate and lactate).

Now let's look at the interactions between *L. lactis* and *A. fabarum*:

![fig5e2](images/fig5e2.png)

Can you inspect what is donated from *L. lactis* to *A. fabarum* ? 

In [ ]:
# type your code here... 

Or click below to reveal the solution:

In [ ]:

df.query('donor == "L_lactis" and receiver == "A_fabarum" and frequency > 0.1')

L. lactis is expected to donate lactate, GABA and some amino acids to A. fabarum. In the simulations we observe the exchange of GABA (M_4abut_e). We also observe exchange of glycerol and arginine, although these were not observed experimentally.

No exchange was observed experimentally from *A. fabarum* to *L. lactis*, but lets see if some are predicted in the simulations:

In [ ]:
# type your code here... 

In [ ]:
Or click below to reveal the solution:

In [ ]:

df.query('donor == "A_fabarum" and receiver == "L_lactis" and frequency > 0.1')

## 2.2 Building an interaction network

Now we will use the [**pyvis**](https://pyvis.readthedocs.io/en/latest/) library to display a network with all the cross-feeding interactions.

Actually, let's start by selecting only:

- interactions that have a frequency above 10% and a mass rate below 0.001 g/gDW/h
- interactions between species (not from/to the environment)


In [ ]:
# type your code here...

# selected = df.query("...")

Or click below to reveal the solution:

In [ ]:

selected = df.query("frequency > 0.1 and mass_rate > 0.001 and donor != 'environment' and receiver != 'environment'")

Building the network is simple, but a bit too much for this tutorial, so we have *"cooked it up"* for you:

In [ ]:
from pyvis.network import Network

net = Network(directed=True, notebook=True, height='500px', width='800px')

species = set(selected['donor']) | set(selected['receiver'])
net.add_nodes(species)

for cpd in set(selected['compound']):
    net.add_node(cpd, shape='box')

for _, row in selected.iterrows():
    net.add_edge(row['donor'], row['compound'], value=row['mass_rate'])
    net.add_edge(row['compound'], row['receiver'], value=row['mass_rate'])

net.show('network.html')

You can try dragging around the nodes in the network for better visualization. 

> If the compound identifiers are hard to interpret, you can look them up in the [BiGG database](http://bigg.ucsd.edu/).

---------

Congratulations, you reached the end of the tutorial. 🥳

If you finished early, feel free to ask questions or try to help another colleague... 🙂